In [1]:
!pip install -U transformers datasets scipy scikit-learn ipywidgets

Looking in indexes: https://pypi.org/simple, http://kakaobrain-pypi.dev.9rum.cc/
     |████████████████████████████████| 2.5 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 237 kB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 27.2 MB 62 kB/s  eta 0:00:012    |▍                               | 296 kB 11.3 MB/s eta 0:00:03     |█                               | 788 kB 11.3 MB/s eta 0:00:03
     |████████████████████████████████| 24.9 MB 241 kB/s  eta 0:00:01
     |████████████████████████████████| 121 kB 75.5 MB/s eta 0:00:01
     |████████████████████████████████| 733 kB 74.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 76.2 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 78.3 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 67.7 MB/s eta 0:00:01
     |████████████████████████████████| 114 kB 75.9 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 10.1 MB/s eta 0:00:01
     |███████████████████

필요한 라이브러리를 임포트합니다.

In [21]:
import random
from IPython.display import display, HTML

import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

태스크와 모델을 정의합니다.

In [22]:
task = "nli"
model_checkpoint = "klue/roberta-base"
batch_size = 32

HuggingFace `datasets` 라이브러리에 등록된 KLUE 데이터셋 중, NLI 데이터를 내려받습니다.

In [23]:
datasets = load_dataset("klue", task)

Reusing dataset klue (/root/.cache/huggingface/datasets/klue/nli/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90)


데이터

In [24]:
datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'hypothesis', 'label', 'premise', 'source'],
        num_rows: 24998
    })
    validation: Dataset({
        features: ['guid', 'hypothesis', 'label', 'premise', 'source'],
        num_rows: 3000
    })
})

In [25]:
datasets["train"][0]

{'guid': 'klue-nli-v1_train_00000',
 'hypothesis': '힛걸 진심 최고로 멋지다.',
 'label': 0,
 'premise': '힛걸 진심 최고다 그 어떤 히어로보다 멋지다',
 'source': 'NSMC'}

In [26]:
metric = load_metric("glue", "mnli")

In [27]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

In [28]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []
    
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])

    display(HTML(df.to_html()))

In [29]:
show_random_elements(datasets["train"])

,guid,hypothesis,label,premise,source
0,klue-nli-v1_train_02940,두 차례에 걸쳐 구매 수량을 확대하니 수급 상황이 불안정해진다.,contradiction,그간 두 차례에 걸친 구매 수량의 확대에도 불구하고 안정적인 수급 상황이 계속되고 있다.,policy
1,klue-nli-v1_train_02270,일본 만화의 출판 과정은 한국과 일본이 똑같다.,neutral,"과거에 일본 만화의 한국어판은 인쇄 과정에서 그림이 반전되어 좌철 방식으로 출판되었으나, 90년대 초중반부터는 우철 방식 그대로 출판하는 것이 일반적이다.",wikipedia
2,klue-nli-v1_train_21472,액션의 합은 완벽하다.,contradiction,최소한의 합도 맞지 않는 액션이 주는 피곤함.,NSMC
3,klue-nli-v1_train_03182,남학생은 자가가 없다.,entailment,그들은 임대아파트에 사는 한 남학생 집을 방문했다.,wikitree
4,klue-nli-v1_train_04149,인천시에서는 27일과 28일 각각 1백 명이 넘는 확진자가 발생하였다.,neutral,기사에 언급된 보도자료는 인천시에서 26일 발생한 61명의 확진자에 대한 내용이었다.,wikinews
5,klue-nli-v1_train_07692,만화책으로 보신분께는 추천하지 않습니다.,entailment,만화책으로 최신까지 보신분께는 비추드립니다,NSMC
6,klue-nli-v1_train_19614,주변에 슈퍼마켓을 찾을 수 없어요.,contradiction,주변에 수퍼마켓도 많았고 에펠탑까지의 거리도 가까워요.,airbnb
7,klue-nli-v1_train_22583,조안은 집안의 반대로 더이상 과학을 추구하지 않는다.,neutral,파인만 집안의 어머니와 할머니는 조안이 과학을 추구하는 것을 못 하게 말렸다.,wikipedia
8,klue-nli-v1_train_12550,직접민주주의 체제의 예시로 아테네 민주주의가 있다.,entailment,아테네 민주주의는 직접민주주의 체제로서 입법과 행정에 대한 결정은 유권자의 투표에 의해 결정되었다.,wikipedia
9,klue-nli-v1_train_19277,위치가 나쁘지 않습니다.,entailment,제일 좋았던 건 여행하기 최고의 위치였습니다.,airbnb


In [30]:
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))

In [31]:
fake_preds, fake_labels

(array([0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
        1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0]),
 array([0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
        0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1]))

In [32]:
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.578125}

In [33]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [34]:
tokenizer("힛걸 진심 최고다 그 어떤 히어로보다 멋지다")

{'input_ids': [2, 1, 7254, 3841, 2062, 636, 3711, 12717, 2178, 2062, 11980, 2062, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [35]:
task_to_keys = {
    "ynat": ("title", None),
    "nli": ("premise", "hypothesis"),
    "sts": ("sentence1", "sentence2"),
}

In [36]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {datasets['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {datasets['train'][0][sentence1_key]}")
    print(f"Sentence 2: {datasets['train'][0][sentence2_key]}")

Sentence 1: 힛걸 진심 최고다 그 어떤 히어로보다 멋지다
Sentence 2: 힛걸 진심 최고로 멋지다.


In [57]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True, return_token_type_ids=False)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True, return_token_type_ids=False)

In [58]:
preprocess_function(datasets['train'][:5])

{'input_ids': [[2, 1, 7254, 3841, 2062, 636, 3711, 12717, 2178, 2062, 11980, 2062, 3, 1, 7254, 3841, 2200, 11980, 2062, 18, 3], [2, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 3, 3911, 2377, 2366, 1525, 2062, 18, 3], [2, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 3, 1282, 2955, 3308, 2052, 3944, 11580, 2359, 2062, 18, 3], [2, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 3, 3911, 2377, 2366, 5105, 2318, 831, 717, 2886, 2069, 575, 555, 2062, 18, 3], [2, 10522, 2548, 2500, 6328, 2170, 6189, 5916, 4015, 2116, 1039, 2219, 3606, 18, 3, 10522, 2548, 2500, 6328, 27135, 5916, 4015, 1642, 2015, 2259, 4258, 2219, 3606, 18, 3]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1

In [59]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/klue/nli/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90/cache-6178fd4ad8e57438.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/klue/nli/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90/cache-4e8732ecf382b037.arrow


In [70]:
encoded_datasets["train"][2]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'guid': 'klue-nli-v1_train_00002',
 'hypothesis': '소닉붐이 정말 멋있었다.',
 'input_ids': [2,
  3911,
  2377,
  2366,
  1521,
  3061,
  4785,
  1282,
  2955,
  3308,
  3515,
  2170,
  22,
  2532,
  5675,
  3,
  1282,
  2955,
  3308,
  2052,
  3944,
  11580,
  2359,
  2062,
  18,
  3],
 'label': 1,
 'premise': '100분간 잘껄 그래도 소닉붐땜에 2점준다',
 'source': 'NSMC'}

In [41]:
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier

In [60]:
metric_name = "accuracy"

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [61]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [62]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [63]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.616500,0.449497,0.057000
2,0.302800,0.415396,0.040333


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()